In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

user_login = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "user_login") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

user_login.show()

+-------+----------+
|user_id|login_date|
+-------+----------+
|      1|2024-03-01|
|      1|2024-03-02|
|      1|2024-03-03|
|      1|2024-03-04|
|      1|2024-03-06|
|      1|2024-03-10|
|      1|2024-03-11|
|      1|2024-03-12|
|      1|2024-03-13|
|      1|2024-03-14|
|      1|2024-03-20|
|      1|2024-03-25|
|      1|2024-03-26|
|      1|2024-03-27|
|      1|2024-03-28|
|      1|2024-03-29|
|      1|2024-03-30|
|      2|2024-03-01|
|      2|2024-03-02|
|      2|2024-03-03|
+-------+----------+
only showing top 20 rows



In [3]:
user_login.createOrReplaceTempView("user_login")

spark.sql('''
with sample_cte as (
select
    distinct
    user_login.user_id,
    user_login.login_date
from
    user_login),
sample_cte2 as (
select
    *,
    day(login_date)-row_number() over (partition by user_id order by login_date) as rn
from
    sample_cte),
sample_cte3 as (
select
    *,
    count(rn) over (partition by rn,user_id) as cnt
from
    sample_cte2)
select
    user_id,
    min(login_date) as start_date,
    max(login_date) as end_date,
    cnt
from
    sample_cte3
where
    cnt >=5
group by user_id,cnt
''').show()

+-------+----------+----------+---+
|user_id|start_date|  end_date|cnt|
+-------+----------+----------+---+
|      3|2024-03-01|2024-03-05|  5|
|      1|2024-03-10|2024-03-14|  5|
|      1|2024-03-25|2024-03-30|  6|
+-------+----------+----------+---+

